In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer

imputer_knn = KNNImputer(n_neighbors=5)

train_num = training_set.select_dtypes(include=[np.number])
valid_num = validation_set.select_dtypes(include=[np.number])

train_cat = training_set.select_dtypes(include=object)
valid_cat = validation_set.select_dtypes(include=object)

SimpleImputer(strategy='most_frequent').fit_transform(train_cat)
SimpleImputer(strategy='most_frequent').fit_transform(valid_cat)


training_imputed[train_num]  = KNNImputer(n_neighbors=5).fit_transform(train_num)
validation_imputed[valid_num] = KNNImputer(n_neighbors=5).fit_transform(valid_num)

In [ ]:
train_set = train_set.drop(columns={'Country','Thinness 10-19 Years','Under-Five Deaths','Schooling'})
test_set = test_set.drop(columns={'Country','Thinness 10-19 Years','Under-Five Deaths','Schooling'})

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        return X.drop(columns=self.columns, axis=1)

# Create an instance of the transformer
dropper = ColumnDropper(columns=['Country','Thinness 10-19 Years','Under-Five Deaths','Schooling'])

# Apply the transformer to a DataFrame
#X_transformed = dropper.transform(X)


In [ ]:
# Imputation and Standardization

num_pipeline = Pipeline([
    ("dropper", ColumnDropper(columns=['Thinness 10-19 Years','Under-Five Deaths','Schooling'])),
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler())
])

cat_pipeline = Pipeline([
    ("dropper", ColumnDropper(columns=['Country'])),
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("label_encoder", FunctionTransformer(LabelEncoder().fit_transform)),
    ("reshape", FunctionTransformer(lambda x: x.reshape(-1,1), validate=False))
])

In [ ]:
X1 = X_train[['Infant Deaths']]
X2 = X_valid[['Infant Deaths']]

lr_temp = make_pipeline(preprocessing, LinearRegression())
lr_temp.fit(X1,y_train)


y_valid_pred = lr_temp.predict(X_valid_selected) 

print("Mean absolute error =", round(mean_absolute_error(y_valid, y_valid_pred), 2)) 
print("Mean squared error =", round(mean_squared_error(y_valid, y_valid_pred), 2)) 
print("Median absolute error =", round(median_absolute_error(y_valid, y_valid_pred), 2)) 
print("Explain variance score =", round(explained_variance_score(y_valid, y_valid_pred), 2)) 
print("R2 score =", round(r2_score(y_valid, y_valid_pred), 2))


Mean absolute error = 2.84
Mean squared error = 15.62
Median absolute error = 1.95
Explain variance score = 0.84
R2 score = 0.84


In [ ]:
w = X_train['Status']
w_multi = np.array([0 if x=='Developing' else 1 for x in w])

w_v = X_valid['Status']
w_multi_v = np.array([0 if x=='Developing' else 1 for x in w_v])

In [ ]:
# Ready-to-use S-Learner using LinearRegression
learner_s = LRSRegressor(control_name='control')
learner_s.fit(X_c, w_multi, y_c)
pred_s = learner_s.predict(X_v, w_multi_v)
error_s = mean_squared_error(pred_s, y_v)**(1/2) #np.mean(np.abs(pred_s - y_v))
print('RMSE using S-Learner with LinearRegression: {}'.format(round(error_s,5)))

# Ready-to-use T-Learner using XGBTRegressor
learner_t = BaseTRegressor(learner=XGBRegressor(),control_name='control')
learner_t.fit(X_c, w_multi, y_c)
pred_t = learner_t.predict(X_v, w_multi_v)
error_t = mean_squared_error(pred_t, y_v)**(1/2) #np.mean(np.abs(pred_t - y_v))
print('RMSE using T-Learner with XGBRegressor: {}'.format(round(error_t,5)))

# X Learner without propensity score input
learner_x = BaseXRegressor(learner=XGBRegressor(),control_name='control')
learner_x.fit(X_c, w_multi, y_c)
pred_x = learner_x.predict(X_v, w_multi_v)
error_x = mean_squared_error(pred_x, y_v)**(1/2) #np.mean(np.abs(pred_x - y_v))
print('RMSE using X-Learner with XGBRegressor: {}'.format(round(error_x,5)))

# R Learner without propensity score input
learner_r = BaseRRegressor(learner = XGBRegressor(),control_name='control')
learner_r.fit(X_c, w_multi, y_c)
pred_r = learner_r.predict(X_v, w_multi_v)
error_r = mean_squared_error(pred_r, y_v)**(1/2) #np.mean(np.abs(pred_r - y_v))
print('RMSE using R-Learner with XGBRegressor: {}'.format(round(error_r),5))


RMSE using S-Learner with LinearRegression: 66.05561
RMSE using T-Learner with XGBRegressor: 63.86639
RMSE using X-Learner with XGBRegressor: 67.71821
RMSE using R-Learner with XGBRegressor: 71


The Root Mean Squared Error (RMSE) is a common measure of the performance of a regression model. A lower RMSE value indicates a better fit of the model to the data.

Based on the results, we can conclude that the T-Learner with XGBRegressor has the best fit to the data. 